<a href="https://colab.research.google.com/github/kylemccullough1/MachineLearning/blob/main/Assignment_4_Problem_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [187]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Load all the data
dataset = pd.ExcelFile("/content/drive/MyDrive/Assignment_4/Asssignment4_Data.xlsx")
# I renamed the sheets to be Train and Test
train_data = pd.read_excel(dataset, 'Train')
test_data = pd.read_excel(dataset, 'Test')
train_data, test_data = train_data.drop(columns=['House ID']), test_data.drop(columns=['House ID'])
# Cleaning data to make it easier for computations
train_data['Local Price'] = train_data['Local Price'].apply(lambda x: round(x))
train_data['Living area'] = train_data['Living area'].apply(lambda x: round(x))
train_data['Land Area'] = train_data['Land Area'].apply(lambda x: round(x))
train_data['Age of home'] = train_data['Age of home'].apply(lambda x: x - (x % 10))
train_data

,Local Price,Bathrooms,Land Area,Living area,# Garages,# Rooms,# Bedrooms,Age of home,Construction type
0,5,1.0,3,1,1.0,7,4,40,Apartment
1,5,1.0,4,2,2.0,7,4,60,House
2,5,1.0,2,1,1.0,6,3,40,Condo
3,5,1.0,4,1,1.0,6,3,50,Apartment
4,5,1.0,4,1,1.0,6,3,40,Apartment
5,4,1.0,4,1,1.0,6,3,50,Condo
6,6,1.0,6,1,1.0,7,3,50,Condo
7,6,1.0,10,2,0.0,6,3,30,House
8,16,2.5,10,3,2.0,10,5,40,Condo
9,14,2.5,13,3,2.0,9,5,10,Apartment


In [3]:
# Probability distribution for Local Price (+1 for all 0s to handle zero frequency problem). I got these by counting every apartment that had 4,5,6, 7, 8,
# 9, 14, and 16 for apartment, house, and condo classes
local_price_probabilities = {
    'Apartment': {
        4: 1/7,
        5: 5/7,
        6: 1/7,
        7: 1/7,
        8: 2/7,
        9: 2/7,
        14: 2/7,
        16: 1/7
    },
    'Condo': {
        4: 2/6,
        5: 2/6,
        6: 3/6,
        7: 1/6,
        8: 2/6,
        9: 1/6,
        14: 1/6,
        16: 2/6
    },
    'House': {
        4: 1/7,
        5: 3/7,
        6: 5/7,
        7: 2/7,
        8: 1/7,
        9: 1/7,
        14: 1/7,
        16: 1/7
    },
    4: 4/20,
    5: 10/20,
    6: 9/20,
    7: 4/20,
    8: 5/20,
    9: 4/20,
    14: 4/20,
    16: 4/20
}
# Probability distribution for Bathrooms (+1 for all 0s to handle zero frequency problem). I got these by counting every apartment that had 1.0, 1.5, and 2.5 bathrooms, 
# and then counting every house that had 1.0, 1.5, and 2.5 bathrooms, and finally by counting how many condos had 1.0, 1.5, and 2.5 bathrooms.
#               1.0.  1.5.  2.5
#   Apartment:  6/7  2/7  2/7
#   Condo:      5/6   2/6   2/6
#   House:      7/7.  2/7   1/7
bathroom_probabilities = {
  'Apartment': {
      1.0: 6/7,
      1.5: 2/7,
      2.5: 2/7
  },
  'Condo': {
      1.0: 5/6,
      1.5: 2/6,
      2.5: 2/6
  },
  'House': {
      1.0: 7/7,
      1.5: 2/7,
      2.5: 1/7
  },
  1.0: 18/20,
  1.5: 6/20,
  2.5: 5/20
}
land_area_probabilities = {
    'Apartment': {
        2: 1/7,
        3: 2/7,
        4: 3/7,
        5: 2/7,
        6: 1/7,
        7: 1/7,
        8: 2/7,
        10: 1/7,
        13: 2/7
    },
    'Condo': {
        2: 2/6,
        3: 1/6,
        4: 2/6,
        5: 1/6,
        6: 2/6,
        7: 3/6,
        8: 1/6,
        10: 2/6,
        13: 1/6
    },
    'House': {
        2: 1/7,
        3: 1/7,
        4: 2/7,
        5: 2/7,
        6: 3/7,
        7: 2/7,
        8: 1/7,
        10: 3/7,
        13: 1/7
    },
    2: 4/20,
    3: 4/20,
    4: 7/20,
    5: 5/20,
    6: 6/20,
    7: 6/20,
    8: 4/20,
    10: 6/20,
    13: 4/20
}
living_area_probabilities = {
    'Apartment': {
      1: 5/7,
      2: 3/7,
      3: 2/7  
    },
    'Condo': {
        1: 6/6,
        2: 1/6,
        3: 2/6
    },
    'House': {
        1: 2/7,
        2: 5/7,
        3: 1/7
    },
    1: 15/20,
    2: 9/20,
    3: 5/20
}
garages_probabilities = {
    'Apartment': {
        0.0: 2/7,
        1.0: 4/7,
        1.5: 2/7,
        2.0: 3/7
    },
    'Condo': {
        0.0: 1/6,
        1.0: 5/6,
        1.5: 1/6,
        2.0: 3/6
    },
    'House': {
        0.0: 3/7,
        1.0: 3/7,
        1.5: 3/7,
        2.0: 2/7
    },
    0.0: 6/20,
    1.0: 12/20,
    1.5: 6/20,
    2.0: 8/20
}
room_probabilities = {
    'Apartment': {
        5: 2/7,
        6: 3/7,
        7: 3/7,
        8: 2/7,
        9: 2/7,
        10: 1/7
    },
    'Condo': {
        5: 1/6,
        6: 5/6,
        7: 2/6,
        8: 1/6,
        9: 3/6,
        10: 2/6
    },
    'House': {
        5: 2/7,
        6: 5/7,
        7: 3/7,
        8: 1/7,
        9: 1/7,
        10: 1/7
    },
    5: 6/20,
    6: 6/20,
    7: 12/20,
    8: 6/20,
    9: 8/20,
    10: 8/20
}
bedroom_probabilities = {
    'Apartment': {
      2: 2/7,
      3: 4/7,
      4: 3/7,
      5: 2/7  
    },
    'Condo': {
        2: 1/6,
        3: 6/6,
        4: 1/6,
        5: 2/6
    },
    'House': {
        2: 2/7,
        3: 6/7,
        4: 2/7,
        5: 1/7
    },
    2: 5/20,
    3: 16/20,
    4: 6/20,
    5: 5/20
}
age_probabilities = {
    'Apartment': {
        10: 2/7,
        20: 2/7,
        30: 1/7,
        40: 4/7,
        50: 3/7,
        60: 1/7 
    },
    'Condo': {
        10: 2/6,
        20: 1/6,
        30: 2/6,
        40: 3/6,
        50: 3/6,
        60: 1/6 
    },
    'House': {
        10: 1/7,
        20: 2/7,
        30: 6/7,
        40: 1/7,
        50: 1/7,
        60: 2/7
    },
    10: 5/20,
    20: 5/20,
    30: 9/20,
    40: 8/20,
    50: 7/20,
    60: 4/20
}

In [4]:
P_apartment = 7/20
P_condo = 6/20
P_house = 7/20

In [186]:
X_train = train_data.drop([train_data.columns[-1]], axis = 1)
y_train = train_data[train_data.columns[-1]]
test = test_data.drop([test_data.columns[-1]], axis = 1)
y_test = test_data[test_data.columns[-1]]

In [167]:
import collections
import bisect
  

def closest(dict, residence, value, key):
  if(residence == 'None'):
      res = bisect.bisect_left(list(dict.keys())[3:], value)
      if(res > len(dict) - 1):
        return list(dict)[len(dict) - 1]
      else:
        return list(dict)[3:][res]
  else:
    res = bisect.bisect_left(list(dict[residence].keys()), value)
    if(key):
      if(res > len(dict[residence]) - 1):
        return list(dict[residence])[len(dict[residence]) - 1]
      else:
        return list(dict[residence])[res]
    else:
      if(res > len(dict[residence]) - 1):
        return dict[residence][list(dict[residence])[len(dict[residence]) - 1]]
      else:
        return dict[residence][list(dict[residence])[res]]
  
  


In [184]:
test['Local Price'] = test['Local Price'].apply(lambda x: round(x))
test['Living area'] = test['Living area'].apply(lambda x: round(x))
test['Land Area'] = test['Land Area'].apply(lambda x: round(x))
test['Age of home'] = test['Age of home'].apply(lambda x: x - (x % 10))

In [178]:
def posterior(row, residence):
  if(residence == 'Apartment'):
    local_probA = closest(local_price_probabilities, 'Apartment', row['Local Price'], False)
    bathroom_probA = closest(bathroom_probabilities, 'Apartment', row['Bathrooms'], False)
    land_probA = closest(land_area_probabilities, 'Apartment', row['Land Area'], False)
    living_probA = closest(living_area_probabilities, 'Apartment', row['Living area'], False)
    garage_probA = closest(garages_probabilities, 'Apartment', row['# Garages'], False)
    room_probA = closest(room_probabilities, 'Apartment', row['# Rooms'], False)
    bedroom_probA = closest(bedroom_probabilities, 'Apartment', row['# Bedrooms'], False)
    age_probA = closest(age_probabilities, 'Apartment', row['Age of home'], False)

    local_prob = local_price_probabilities[closest(local_price_probabilities, 'None', row['Local Price'], True)]
    bathroom_prob = bathroom_probabilities[closest(bathroom_probabilities, 'None', row['Bathrooms'], True)]
    land_prob = land_area_probabilities[closest(land_area_probabilities, 'None', row['Land Area'], True)]
    living_prob = living_area_probabilities[closest(living_area_probabilities, 'None', row['Living area'], True)]
    garage_prob = garages_probabilities[closest(garages_probabilities, 'None', row['# Garages'], True)]
    room_prob =  room_probabilities[closest(room_probabilities, 'None', row['# Rooms'], True)]
    bedroom_prob = bedroom_probabilities[closest(bedroom_probabilities, 'None', row['# Bedrooms'], True)]
    age_prob = age_probabilities[closest(age_probabilities, 'None', row['Age of home'], True)]

    return (local_probA * bathroom_probA * land_probA * living_probA * garage_probA * room_probA * bedroom_probA * age_probA * P_apartment) / (local_prob * bathroom_prob * land_prob * living_prob * garage_prob * room_prob * bedroom_prob * age_prob)
  elif(residence == 'Condo'):
    local_probA = closest(local_price_probabilities, 'Condo', row['Local Price'], False)
    bathroom_probA = closest(bathroom_probabilities, 'Condo', row['Bathrooms'], False)
    land_probA = closest(land_area_probabilities, 'Condo', row['Land Area'], False)
    living_probA = closest(living_area_probabilities, 'Condo', row['Living area'], False)
    garage_probA = closest(garages_probabilities, 'Condo', row['# Garages'], False)
    room_probA = closest(room_probabilities, 'Condo', row['# Rooms'], False)
    bedroom_probA = closest(bedroom_probabilities, 'Condo', row['# Bedrooms'], False)
    age_probA = closest(age_probabilities, 'Condo', row['Age of home'], False)

    local_prob = local_price_probabilities[closest(local_price_probabilities, 'None', row['Local Price'], True)]
    bathroom_prob = bathroom_probabilities[closest(bathroom_probabilities, 'None', row['Bathrooms'], True)]
    land_prob = land_area_probabilities[closest(land_area_probabilities, 'None', row['Land Area'], True)]
    living_prob = living_area_probabilities[closest(living_area_probabilities, 'None', row['Living area'], True)]
    garage_prob = garages_probabilities[closest(garages_probabilities, 'None', row['# Garages'], True)]
    room_prob = room_probabilities[closest(room_probabilities, 'None', row['# Rooms'], True)]
    bedroom_prob = bedroom_probabilities[closest(bedroom_probabilities, 'None', row['# Bedrooms'], True)]
    age_prob = age_probabilities[closest(age_probabilities, 'None', row['Age of home'], True)]

    return (local_probA * bathroom_probA * land_probA * living_probA * garage_probA * room_probA * bedroom_probA * age_probA * P_condo) / (local_prob * bathroom_prob * land_prob * living_prob * garage_prob * room_prob * bedroom_prob * age_prob)
  elif(residence == 'House'):
    local_probA = closest(local_price_probabilities, 'House', row['Local Price'], False)
    bathroom_probA = closest(bathroom_probabilities, 'House', row['Bathrooms'], False)
    land_probA = closest(land_area_probabilities, 'House', row['Land Area'], False)
    living_probA = closest(living_area_probabilities, 'House', row['Living area'], False)
    garage_probA = closest(garages_probabilities, 'House', row['# Garages'], False)
    room_probA = closest(room_probabilities, 'House', row['# Rooms'], False)
    bedroom_probA = closest(bedroom_probabilities, 'House', row['# Bedrooms'], False)
    age_probA = closest(age_probabilities, 'House', row['Age of home'], False)
    
    local_prob = local_price_probabilities[closest(local_price_probabilities, 'None', row['Local Price'], True)]
    bathroom_prob = bathroom_probabilities[closest(bathroom_probabilities, 'None', row['Bathrooms'], True)]
    land_prob = land_area_probabilities[closest(land_area_probabilities, 'None', row['Land Area'], True)]
    living_prob = living_area_probabilities[closest(living_area_probabilities, 'None', row['Living area'], True)]
    garage_prob = garages_probabilities[closest(garages_probabilities, 'None', row['# Garages'], True)]
    room_prob = room_probabilities[closest(room_probabilities, 'None', row['# Rooms'], True)]
    bedroom_prob = bedroom_probabilities[closest(bedroom_probabilities, 'None', row['# Bedrooms'], True)]
    age_prob = age_probabilities[closest(age_probabilities, 'None', row['Age of home'], True)]
    return (local_probA * bathroom_probA * land_probA * living_probA * garage_probA * room_probA * bedroom_probA * age_probA * P_house) / (local_prob * bathroom_prob * land_prob * living_prob * garage_prob * room_prob * bedroom_prob * age_prob)

In [185]:
for x in range(test.shape[0]):
  apartment = posterior(test.iloc[x], 'Apartment')
  condo = posterior(test.iloc[x], 'Condo')
  house = posterior(test.iloc[x], 'House')
  if(apartment >= condo and apartment >= house):
    print('Apartment', apartment)
    print('Condo', condo)
    print('House', house)
    print('---Final Classification: Apartment')
  elif(condo >= apartment and condo >= house):
    print('Apartment', apartment)
    print('Condo', condo)
    print('House', house)
    print('---Final Classification: Condo')
  else:
    print('Apartment', apartment)
    print('Condo', condo)
    print('House', house)
    print('---Final Classification: House')


Apartment 0.06662637469969884
Condo 1.37818097399335
House 0.5205185523413974
---Final Classification: Condo
Apartment 0.3597824233783737
Condo 0.08820358233557438
House 0.049969781024774146
---Final Classification: Apartment
Apartment 0.32380418104053643
Condo 0.13230537350336158
House 0.07870240511401927
---Final Classification: Apartment
Apartment 0.08883516626626516
Condo 0.021778662305080097
House 0.4441758313313258
---Final Classification: House
Apartment 0.17989121168918693
Condo 0.7938322410201696
House 0.35978242337837385
---Final Classification: Condo
